In [1]:
import pandas as pd

import numpy as np

from sklearn import datasets

from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVC

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import KFold

In [2]:
import json
import gc
import random
i = 0
reviews_lst_good= []
reviews_lst_bad = []
count_good_comments = 0
count_bad_comments = 0

# G:\\Мой диск\\Work\\Experiment\\Electronics_5.json
# /Volumes/GoogleDrive/Мой диск/Work/Experiment/Electronics_5.json
with open('G:\\Мой диск\\Work\\Experiment\\Electronics_5.json', 'r') as f: # указать свой путь  /Volumes/GoogleDrive/Мой диск/Work/Experiment/Books_5.json
        for line in f:
            if json.loads(line)['overall'] > 3:
                  count_good_comments += 1
            elif json.loads(line)['overall'] < 3:
                  count_bad_comments += 1
            
print (count_good_comments)
print (count_bad_comments)

1356067
190864


In [3]:
size_of_df = 5000 # min (count_good_comments, count_bad_comments)
print (size_of_df)

with open('G:\\Мой диск\\Work\\Experiment\\Electronics_5.json', 'r') as f: #/Volumes/GoogleDrive/Мой диск/Work/Experiment/Books_5.json
        for line in f:
            if json.loads(line)['overall'] > 3 and len(reviews_lst_good)<size_of_df:
                reviews_lst_good.append (json.loads(line))
            
            elif json.loads(line)['overall'] < 3 and len(reviews_lst_bad)<size_of_df:
                reviews_lst_bad.append (json.loads(line))

            if len(reviews_lst_good) + len (reviews_lst_bad) == 10000:
                break

random.shuffle(reviews_lst_good)
random.shuffle(reviews_lst_bad)
       

5000


In [4]:
print (len(reviews_lst_good))
print (len(reviews_lst_bad))

5000
5000


In [5]:
reviewerID_lst = []
reviewerName_lst = []
reviewText_lst = []
overall_lst = []
summary_lst = []

# reviews_lst [0]['overall']<5 #['asin']for 

for i in reviews_lst_good:
    reviewerID_lst.append (i['reviewerID'])
    reviewText_lst.append (i['reviewText'])
    overall_lst.append (i['overall'])
    summary_lst.append (i['summary'])
    
for i in reviews_lst_bad:
    reviewerID_lst.append (i['reviewerID'])
    reviewText_lst.append (i['reviewText'])
    overall_lst.append (i['overall'])
    summary_lst.append (i['summary'])
    
df = pd.DataFrame ()

df['reviewerID'] = reviewerID_lst
# df['reviewerName'] = reviewerName_lst
df['reviewText'] = reviewText_lst
df['overall'] = overall_lst
df['summary'] = summary_lst

In [6]:
df ['mark'] = -1
df.loc[df['overall'] > 3,'mark'] = 1

print (len(df[df['mark'] > 0]))
print (len(df[df['mark'] < 0]))

5000
5000


In [7]:
df[(df['mark'] != 0)&(df['mark'] != 1)]

,reviewerID,reviewText,overall,summary,mark
5000,A39AEX2G3SVZ8L,I am new to digital photography and was wary o...,2.0,[Inexpensive] but Limited Digital Camara,-1
5001,A2P5U7BDKKT7FW,The cable is very wobbly and sometimes disconn...,2.0,Cheap proprietary scam,-1
5002,A19GDOQK033YCM,I'm not exactly a headphone connoisseur but I ...,2.0,Very poor construction,-1
5003,A5JLAU2ARJ0BO,I bought the Coolpad after reading the positiv...,1.0,Does not seem to cool down my laptop,-1
5004,A227BIJ309IRM5,I bought two sets of these sensors and install...,2.0,Works ok for a while.,-1
5005,A35T6E4LDUG68M,Breaks and cracks easily. Too thin for protect...,1.0,Breaks easily.,-1
5006,A1GTHUO4BOBWCR,I wrote a review on this DVD player 2 weeks ag...,2.0,"Worked great for 8 months, but now it's giving...",-1
5007,A1GUJ2DDBVMYDA,Good product but will not connect with Canon 7...,1.0,not compatible with 7d.,-1
5008,AFWMVMPB2WVP9,This item was such a HUGE disappointment. Init...,1.0,Poor Quality WILL NOT LAST!!!,-1
5009,A4MLAYF8FDJPX,I've had the same problems that others have al...,2.0,Sorta works,-1


In [8]:
reviews_train = list(df[df['overall'] > 3]['reviewText'][:-100]) + list(df[df['overall'] < 3]['reviewText'][:-100])
reviews_test = list(df[df['overall'] > 3]['reviewText'][-100:]) + list(df[df['overall'] < 3]['reviewText'][-100:])

In [9]:
import re
from nltk.corpus import stopwords 

REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

In [24]:
my_stopword_list = set(stopwords.words('english'))
# my_stopword_list

In [10]:
from sklearn import datasets

from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVC

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import KFold

from sklearn.model_selection import train_test_split

In [25]:
y = list(df[df['overall'] > 3]['mark'][:-100]) + list(df[df['overall'] < 3]['mark'][:-100])

X = reviews_train_clean

X_train, X_val, y_train, y_val = train_test_split(
    X, y, train_size = 0.75
)


vectorizer = TfidfVectorizer(stop_words=my_stopword_list) # , max_features = 35000 # создали генератор признаков 

vectorizer.fit_transform(X)

<9800x36546 sparse matrix of type '<class 'numpy.float64'>'
	with 466738 stored elements in Compressed Sparse Row format>

In [26]:
# 36681 --без max features
vectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words={'with', 'had', 'about', 'was', 'same', 'few', 'by', 'yours', 'or', "shouldn't", 'mightn', 'any', "mightn't", 'between', 'now', 'a', 'in', 'do', 'only', 'into', 'hasn', "you'll", 'where', 'nor', 'some', 'have', 'just', 'y', 'hadn', 'each', 'were', 'are', 'ourselves', "hadn't", 'your', 'bu...edn', 'he', 'whom', 'wasn', 'all', 'shan', 'couldn', 'i', 'she', 'be', 'before', 's', 'didn', 'don'},
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [ ]:
# grid = {'C': np.power(10.0, np.arange(-5, 6))}

# cv = KFold(5, shuffle=True, random_state=241) # делим на 5 блоков

# model = SVC(kernel='linear', random_state=241) # устанавливаем линейной ядро у SVM

# gs = GridSearchCV(model, grid, scoring='accuracy', cv=cv) # показываем, чтобы использовал модель нашу 
# # + искал параметр C сам из диапозона, метод для валивации KFold

# gs.fit(vectorizer.transform(X), y)



# C = gs.best_params_.get('C') # достаём оттуда только C
# print (C)

In [27]:
# 4. Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.



model = SVC(kernel='linear', random_state=241, C=1)

model.fit(vectorizer.transform(X), y)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=241,
  shrinking=True, tol=0.001, verbose=False)

In [28]:
# 5. Найдите 10 слов с наибольшим по модулю весом. Они являются ответом на это задание. Укажите их через запятую или

# пробел, в нижнем регистре, в лексикографическом порядке.



words = vectorizer.get_feature_names()

coef = pd.DataFrame(model.coef_.data, model.coef_.indices)

top_words = coef[0].map(lambda w: abs(w)).sort_values(ascending=False).head(10).index.map(lambda i: words[i])

top_words.sort_values ()

# top_words

Index(['best', 'easy', 'excellent', 'great', 'perfect', 'poor', 'return',
       'unfortunately', 'useless', 'works'],
      dtype='object')

In [35]:
from sklearn.metrics import accuracy_score
print ("Accuracy for C=%s: %s" 
               % ( accuracy_score(y_val, model.predict(X_val))))

ValueError: could not convert string to float: 'i had a dvd that kept on not playing every other disc l got from netflix it would say the disc was dirty or scratched i gave any one that didnt play a good clean with these wipes but rearly got any to work maybe if you had dusty cds but l never got any sense that this was cleaning more of a light polish didnt work for me'

In [30]:
model.score()

TypeError: score() missing 2 required positional arguments: 'X' and 'y'

In [5]:
# coding=utf-8
import pandas
import numpy as np
from sklearn import datasets #, grid_search
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold

import sys
sys.path.append("..")
# from shad_util import print_answer

# 1. Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и
# "атеизм" (инструкция приведена выше). Обратите внимание, что загрузка данных может занять несколько минут

newsgroups = datasets.fetch_20newsgroups(subset='all', categories=['alt.atheism', 'sci.space'])
X = newsgroups.data
y = newsgroups.target

# 2. Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам
# вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют
# информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой
# переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на
# момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.

vectorizer = TfidfVectorizer()
vectorizer.fit_transform(X)

# 3. Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с
# линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM,
# и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).

grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(shuffle=True, random_state=241)
model = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(model, grid, scoring='accuracy', cv=cv)
gs.fit(vectorizer.transform(X), y)

C = gs.best_params_.get('C')

# 4. Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.

model = SVC(kernel='linear', random_state=241, C=C)
model.fit(vectorizer.transform(X), y)

# 5. Найдите 10 слов с наибольшим по модулю весом. Они являются ответом на это задание. Укажите их через запятую или
# пробел, в нижнем регистре, в лексикографическом порядке.

words = vectorizer.get_feature_names()
coef = pandas.DataFrame(model.coef_.data, model.coef_.indices)
top_words = coef[0].map(lambda w: abs(w)).sort_values(ascending=False).head(10).index.map(lambda i: words[i])
top_words.sort()
# print_answer(1, ','.join(top_words))

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:426: FutureWarning: You should specify a value for 'n_splits' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(NSPLIT_WARNING, FutureWarning)


TypeError: cannot sort an Index object in-place, use sort_values instead

In [7]:
np.shape (X)

(1786,)